In [1]:
from datasets import load_dataset
import torch
import evaluate
import nltk
from tqdm import tqdm

C:\Users\DuongNgoKien\AppData\Local\anaconda3\envs\api\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
rouge = evaluate.load('rouge')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DuongNgoKien\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
dataset = load_dataset('xsum')

In [4]:
train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]

In [5]:
def process_data(document):
    labels = ""
    l_sentences = []
    for s in document.split("\n"):
        s = nltk.sent_tokenize(s)
        l_sentences.extend(s)
    len_doc = len(l_sentences)
    n_mask = int(len_doc / 5)
    if n_mask == 0: n_mask =1
    ma_rouge = torch.zeros(size=(len_doc, len_doc)) # Create a matrix to save rouge-f1 score between each sentence and the rest of the document
    for i in range(len_doc):
        for j in range(i+1, len_doc):
            m = rouge.compute(predictions=[l_sentences[i]], references=[[l_sentences[j]]])
            ma_rouge[i,j] = m['rouge1']
            ma_rouge[j,i] = m['rouge1']
    mean_rouge = torch.mean(ma_rouge, 1) # calculate mean rouge-f1 score between a sentence and the reset
    _, indexes = torch.topk(mean_rouge, n_mask)
    indexes = torch.sort(indexes, 0).values
    for i in range(indexes.size()[0]):
        masked_sentence = l_sentences[indexes[i]]
        document = document.replace(masked_sentence, "<mask_1>")
        labels += masked_sentence + " "
    return document.replace('\n', ' '), labels

In [6]:
f_text = open("texts.txt", "a", encoding='utf-8')
f_label = open("labels.txt", "a", encoding='utf-8')
for i in tqdm(range(0, 1000)):
    doc, label = process_data(train_texts[i])
    f_text.write(doc + "\n")
    f_label.write(label + "\n")
f_text.close()
f_label.close()

100%|██████████| 500/500 [13:21:22<00:00, 96.16s/it]     
